Lambda School Data Science, Unit 2: Predictive Modeling

# Kaggle Challenge, Module 3


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.

## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

## Provided Code

In [1]:
%%capturea
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [2]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

## Assignment

### Importing

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
import category_encoders as ce
from sklearn.impute import SimpleImputer
from scipy.stats import randint, uniform
from sklearn.model_selection import RandomizedSearchCV

###  Wrangling

In [4]:
def wrangle(x):
    x = x.copy() #to avoid warning
    
    x['latittude'] = x['latitude'].replace(-2e-09,0) #to replace near 0 with 0
    
    cols_with_zero = ['longitude', 'latitude', 'construction_year', 
                       'gps_height', 'population']
    
    for col in cols_with_zero: #Replace zeroes in cols above with NaN
        x[col] = x[col].replace(0, np.nan)
        x[col+'_MISSING'] = x[col].isnull() #adding a missing indicator
    
    duplicates = ['extraction_type_class', 'extraction_type_group', 'quantity_group', 'payment', 'source', 'waterpoint_type_group']
    x = x.drop(columns=duplicates)

    x = x.drop(columns=['id','recorded_by']) # Unusable info

    x['date_recorded'] = pd.to_datetime(x['date_recorded'], infer_datetime_format=True) #convert to DateTime
    # Extract components from date_recorded, then drop the original column
    x['year_recorded'] = x['date_recorded'].dt.year
    x['month_recorded'] = x['date_recorded'].dt.month
    x['day_recorded'] = x['date_recorded'].dt.day
    x = x.drop(columns='date_recorded')

    x['years'] = x['year_recorded'] - x['construction_year']
    x['years_MISSING'] = x['years'].isnull()

    return x

In [5]:
train = wrangle(train)
test = wrangle(test)

In [6]:
target = 'status_group'

In [7]:
X_train = train.drop(columns=target)

In [8]:
y_train = train[target]

### Randomized Search

In [21]:
pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(random_state=42)
)

params = {
    'simpleimputer__strategy': ['mean', 'median'], 
    'randomforestclassifier__n_estimators': randint(50, 500), 
    'randomforestclassifier__max_depth': [5, 10, 15, 20, None], 
    'randomforestclassifier__max_features': uniform(0, 1), 
}

search = RandomizedSearchCV(
    pipeline,
    param_distributions=params,
    n_iter=15,
    cv=5,
    scoring='accuracy',
    verbose=10,
    return_train_score= True,
    n_jobs=-1
)
search.fit(X_train, y_train);

Fitting 5 folds for each of 15 candidates, totalling 75 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   15.5s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  6.3min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed: 11.5min
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed: 16.0min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed: 17.7min
[Parallel(n_jobs=-1)]: Done  68 out of  75 | elapsed: 19.4min remaining:  2.0min
[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed: 24.5min finished


In [22]:
print('Best hyperparameters', search.best_params_)
print('Score:', search.best_score_)

Best hyperparameters {'randomforestclassifier__max_depth': 20, 'randomforestclassifier__max_features': 0.4357474318068272, 'randomforestclassifier__n_estimators': 472, 'simpleimputer__strategy': 'mean'}
Score: 0.8102693602693603


## Kaggle subs

In [24]:
sample_submission['status_group'] = search.predict(test)

In [25]:
sample_submission.to_csv('RandCV1.csv', index=False)

In [28]:
#I forgot to set index to false so one of my submission files didn't work.
to_fix = pd.read_csv('Dtree_sub.csv')

In [31]:
sample_submission['status_group'] = to_fix['status_group']

In [32]:
sample_submission.to_csv('Dtree_sub.csv', index=False)

### Stacking

In [42]:
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

In [44]:
sample_submission.to_csv('sample_submission.csv', index=False)

In [45]:
# Filenames of your submissions you want to ensemble
files = ['random_forest_1.csv', 'big_model_sub.csv', 'Dtree_Sub.csv','Dtree_sub2.csv', 'RandCV1.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)